In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

batch_size = 32
img_height = 224
img_width = 224
num_classes = 4

In [ ]:
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Model, Sequential
from keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, Rescaling

base = Sequential()
base.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
base.add(Convolution2D(64, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(64, (3, 3), activation='relu'))
base.add(MaxPooling2D((2,2), strides=(2,2)))
 
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(128, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(128, (3, 3), activation='relu'))
base.add(MaxPooling2D((2,2), strides=(2,2)))
 
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(256, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(256, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(256, (3, 3), activation='relu'))
base.add(MaxPooling2D((2,2), strides=(2,2)))
 
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(512, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(512, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(512, (3, 3), activation='relu'))
base.add(MaxPooling2D((2,2), strides=(2,2)))
 
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(512, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(512, (3, 3), activation='relu'))
base.add(ZeroPadding2D((1,1)))
base.add(Convolution2D(512, (3, 3), activation='relu'))
base.add(MaxPooling2D((2,2), strides=(2,2)))
 
base.add(Convolution2D(4096, (7, 7), activation='relu'))
base.add(Dropout(0.5))
base.add(Convolution2D(4096, (1, 1), activation='relu'))
base.add(Dropout(0.5))
base.add(Convolution2D(2622, (1, 1)))
base.add(Flatten())
base.add(Activation('softmax'))

base.load_weights('../input/weights/vgg_weights.h5')

for layer in base.layers[:-7]:
    layer.trainable = False

output = Sequential()
output = Convolution2D(num_classes, (1, 1), name='predictions')(base.layers[-4].output)
output = Flatten()(output)
output = Activation('softmax')(output)

model = Model(inputs=base.input, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, featurewise_center=True, 
    featurewise_std_normalization=True)

test_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    '../input/mergedtrain/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '../input/mergedval/val',
    target_size=(224, 244),
    batch_size=32,
    class_mode='categorical')


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./merged.hdf5', save_best_only=True, monitor='val_loss', mode='min')

model.compile(optimizer=keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10,
  callbacks=[earlyStopping, mcp_save]
)

In [ ]:
from vis.visualization import visualize_activation
from vis.utils import utils
import matplotlib.pyplot as plt

ImportError: ignored